# Postgres-Load training set into Postgres

For my minimum viable project, I'm just using the smaller subset of the data and postgres.

In [1]:
import pandas as pd

In [2]:
%load_ext sql

In [3]:
%load_ext watermark
%watermark -iv

pandas 0.24.0



In [4]:
# only need to do this once! 
# !psql -c "create database nb15;"

In [5]:
%sql postgres://localhost/nb15

'Connected: @nb15'

In [6]:
trainset = pd.read_csv('../data/raw/UNSW_NB15-training-set.csv', index_col='id')

In [7]:
trainset.head()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
id,,,,,,,,,,,,,,,,,,,,,
1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,252,...,1,1,0,0,0,1,1,0,Normal,0
2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,62,...,1,2,0,0,0,1,6,0,Normal,0
3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,62,...,1,3,0,0,0,2,6,0,Normal,0
4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,62,...,1,3,1,1,0,2,1,0,Normal,0
5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,254,...,1,40,0,0,0,2,39,0,Normal,0


In [8]:
trainset.shape

(175341, 44)

In [9]:
trainset.nunique()

dur                  74039
proto                  133
service                 13
state                    9
spkts                  480
dpkts                  443
sbytes                7214
dbytes                6660
rate                 76991
sttl                    11
dttl                     6
sload                80885
dload                77474
sloss                  409
dloss                  370
sinpkt               76161
dinpkt               74245
sjit                 77532
djit                 76831
swin                    13
stcpb                75265
dtcpb                75089
dwin                     7
tcprtt               43319
synack               40142
ackdat               37708
smean                 1357
dmean                 1328
trans_depth             11
response_body_len     2386
ct_srv_src              52
ct_state_ttl             5
ct_dst_ltm              50
ct_src_dport_ltm        47
ct_dst_sport_ltm        32
ct_dst_src_ltm          54
is_ftp_login             4
c

In [10]:
trainset.is_sm_ips_ports.value_counts()

0    172579
1      2762
Name: is_sm_ips_ports, dtype: int64

In [11]:
# only need to do this once! 
# %sql persist trainset

 * postgres://localhost/nb15


'Persisted trainset'

In [12]:
%sql select count(*) from trainset;

 * postgres://localhost/nb15
1 rows affected.


count
175341


In [13]:
%sql select * from trainset limit 5

 * postgres://localhost/nb15
5 rows affected.


id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
1,0.121478,tcp,-,FIN,6,4,258,172,74.08749,252,254,14158.94238,8495.365234,0,0,24.2956,8.375,30.177547,11.830604,255,621772692,2202533631,255,0.0,0.0,0.0,43,43,0,0,1,0,1,1,1,1,0,0,0,1,1,0,Normal,0
2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,62,252,8395.112305,503571.3125,2,17,49.915,15.432865,61.426934,1387.77833,255,1417884146,3077387971,255,0.0,0.0,0.0,52,1106,0,0,43,1,1,1,1,2,0,0,0,1,6,0,Normal,0
3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,62,252,1572.271851,60929.23047,1,6,231.875571,102.737203,17179.58686,11420.92623,255,2116150707,2963114973,255,0.111897,0.061458,0.050439,46,824,0,0,7,1,2,1,1,3,0,0,0,2,6,0,Normal,0
4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,62,252,2740.178955,3358.62207,1,3,152.876547,90.235726,259.080172,4991.784669,255,1107119177,1047442890,255,0.0,0.0,0.0,52,64,0,0,1,1,2,1,1,3,1,1,0,2,1,0,Normal,0
5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,254,252,8561.499023,3987.059814,2,1,47.750333,75.659602,2415.837634,115.807,255,2436137549,1977154190,255,0.128381,0.071147,0.057234,53,45,0,0,43,1,2,2,1,40,0,0,0,2,39,0,Normal,0


In [15]:
%sql select attack_cat, count(*) from trainset group by attack_cat;

 * postgres://localhost/nb15
10 rows affected.


attack_cat,count
Analysis,2000
Backdoor,1746
DoS,12264
Exploits,33393
Fuzzers,18184
Generic,40000
Normal,56000
Reconnaissance,10491
Shellcode,1133
Worms,130
